# 1. 优化算法

TensorFlow提供了多种优化算法:
- tf.train.GradientDescentOptimizer
- tf.train.AdadeltaOptimizer
- tf.train.AdagradOptimizer
- tf.train.AdagradDAOptimizer
- tf.train.MomentumOptimizer
- tf.train.AdamOptimizer
- tf.train.FtrlOptimizer
- tf.train.ProximalGradientDescentOptimizer
- tf.train.ProximalAdagradOptimizer
- tf.train.RMSPropOptimizer

参考：https://www.tensorflow.org/api_guides/python/train#Optimizers

其中AdamOptimizer用的较多，其中使用方法如下：
```
# cross_entropy是损失函数
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
```
详细API描述，请参考：https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer


**练习**: 请将下列代码的优化器更换成`AdamOptimizer`，并通过TensorBoard查看准确率。

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 导入数据
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.truncated_normal([in_size, out_size]), name='W')
            
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.constant(0.1, shape=[1, out_size]), name='b')
            
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
            
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
    
    tf.summary.histogram('weights', Weights)
    tf.summary.histogram('biases', biases)
    
    return outputs

with tf.Graph().as_default() as g:
    with tf.name_scope('input'):
        # X: 输入
        X = tf.placeholder(tf.float32, [None, 784], name="X")
        # Y_: 标签
        Y_ = tf.placeholder(tf.float32, [None, 10])

    # L1：200个神经元
    Y1 = add_layer(X, 784, 200, tf.nn.sigmoid)

    # L2：100个神经元
    Y2 = add_layer(Y1, 200, 100, tf.nn.sigmoid)

    # L3: 60个神经元
    Y3 = add_layer(Y2, 100, 60, tf.nn.sigmoid)

    # L4: 30个神经元
    Y4 = add_layer(Y3, 60, 30, tf.nn.sigmoid)

    # L5: 10个神经元
    Ylogits = add_layer(Y4, 30, 10, tf.nn.sigmoid)

    # Output
    Y = tf.nn.softmax(Ylogits)

    # 损失函数
    with tf.name_scope('loss'):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y_, logits=Ylogits))
        
    tf.summary.scalar('cross_entropy', cross_entropy)
        
    # 优化算法
    with tf.name_scope('train'):
        train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
        
    # 计算准确率
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    tf.summary.scalar('accuracy', accuracy)
    
    merged = tf.summary.merge_all()
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 将当前会话中的计算图保存
        writer = tf.summary.FileWriter("logs/", sess.graph)

        for i in range(10000):
            batch_xs, batch_ys = mnist.train.next_batch(100)
            summary, _ = sess.run([merged, train_step], feed_dict={X: batch_xs, Y_: batch_ys})

            writer.add_summary(summary, i)
            
            if i%100 == 0:
                print(accuracy.eval({X: mnist.test.images, Y_: mnist.test.labels}))

/Users/lyon/.pyenv/versions/3.6.4/envs/scikit-learn/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
0.098
0.098
0.098
0.098
0.098
0.0981
0.1002
0.1064
0.1439
0.2036
0.4081
0.7103
0.7328
0.7437
0.7491
0.7622
0.7736
0.7806
0.7862
0.7903
0.7999
0.8049
0.8079
0.8116
0.8161
0.8233
0.8249
0.8289
0.8325
0.8363
0.8383
0.8404
0.8476
0.8472
0.8519
0.8544
0.8547
0.8595
0.8591
0.8616
0.8642
0.8654
0.8661
0.869

# 2. Hyperparameter-学习速率

在机器学习的上下文中，超参数是在开始学习过程之前设置值的参数，而不是通过训练得到的参数数据。通常情况下，需要对超参数进行优化，给学习机选择一组最优超参数，以提高学习的性能和效果。例如在我们的MNIST任务中，AdamOptimizer的学习速率参数就是一个待确定的值。

为了更好的比较不同超参数下模型的性能，一种比较简单的的方法就是得到各个参数下模型的性能，然后进行对比。TensorBoard支持同时显示不同超参数下的数据，只需要将不同参数下产生的TensorBoard数据写入到不同的子目录即可。

下面是一个很简单的例子，实现了$y=x+step$的功能，将不同的step的数据写入到`logs`的不同子目录下

In [1]:
import tensorflow as tf

for step in [k for k in range(10)]:
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32, shape=(), name='x')
        y = x+step
        tf.summary.scalar('y', y)

        merged = tf.summary.merge_all()

        with tf.Session() as sess:
            writer = tf.summary.FileWriter('logs/step_{0}/'.format(step))

            for i in range(100):
                summary, _ = sess.run([merged, y], feed_dict={x: i/100})

                writer.add_summary(summary, i)

                i += 1

/Users/lyon/.pyenv/versions/3.6.4/envs/scikit-learn/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


**练习**: 根据上述代码，我们探索一下在不同学习速率下模型的性能变化。输出的日志文件保存在`logs/adm_lr_{value}/`子目录下，其中`value`是具体的学习速率值。例如lr=0.001，则对应的TensorFlow事件输出到`logs/lr_0.001`目录下 (提示，修改tf.summary.FileWriter的参数)。
学习速率参数在`AdamOptimizer`中指定，其中学习速率取值从0.001开始，直到0.01，步长为0.001

运行完成之后TensorBoard显示效果图如下：
![image.png](http://p811pjpxl.bkt.clouddn.com/16-1.png)

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 导入数据
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.truncated_normal([in_size, out_size]), name='W')
            
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.constant(0.1, shape=[1, out_size]), name='b')
            
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
            
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
    
    tf.summary.histogram('weights', Weights)
    tf.summary.histogram('biases', biases)
    
    return outputs

with tf.Graph().as_default() as g:
    with tf.name_scope('input'):
        # X: 输入
        X = tf.placeholder(tf.float32, [None, 784], name="X")
        # Y_: 标签
        Y_ = tf.placeholder(tf.float32, [None, 10])

    # L1：200个神经元
    Y1 = add_layer(X, 784, 200, tf.nn.sigmoid)

    # L2：100个神经元
    Y2 = add_layer(Y1, 200, 100, tf.nn.sigmoid)

    # L3: 60个神经元
    Y3 = add_layer(Y2, 100, 60, tf.nn.sigmoid)

    # L4: 30个神经元
    Y4 = add_layer(Y3, 60, 30, tf.nn.sigmoid)

    # L5: 10个神经元
    Ylogits = add_layer(Y4, 30, 10, tf.nn.sigmoid)

    # Output
    Y = tf.nn.softmax(Ylogits)

    # 损失函数
    with tf.name_scope('loss'):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y_, logits=Ylogits))
        
    tf.summary.scalar('cross_entropy', cross_entropy)
        
    # 优化算法
    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
        
    # 计算准确率
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    tf.summary.scalar('accuracy', accuracy)
    
    merged = tf.summary.merge_all()
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 将当前会话中的计算图保存
        writer = tf.summary.FileWriter("logs/", sess.graph)

        for i in range(10000):
            batch_xs, batch_ys = mnist.train.next_batch(100)
            summary, _ = sess.run([merged, train_step], feed_dict={X: batch_xs, Y_: batch_ys})

            writer.add_summary(summary, i)
            
            if i%100 == 0:
                print(accuracy.eval({X: mnist.test.images, Y_: mnist.test.labels}))

/Users/lyon/.pyenv/versions/3.6.4/envs/scikit-learn/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
0.1414
0.8695
0.9122
0.9173
0.9263
0.9369
0.937
0.94
0.9432
0.9348
0.9453
0.9277
0.9431
0.9399
0.9454
0.9412
0.9507
0.9482
0.9462
0.9449
0.9453
0.9518
0.948
0.9528
0.9516
0.952
0.9489
0.9521
0.953
0.9442
0.9536
0.9514
0.9536
0.9419
0.9503
0.9573
0.9516
0.9537
0.9545
0.9544
0.9572
0.9447
0.9563
0.9516

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 导入数据
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.truncated_normal([in_size, out_size]), name='W')
            
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.constant(0.1, shape=[1, out_size]), name='b')
            
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
            
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
    
    tf.summary.histogram('weights', Weights)
    tf.summary.histogram('biases', biases)
    
    return outputs

for lr in [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01]:
    with tf.Graph().as_default() as g:
        with tf.name_scope('input'):
            # X: 输入
            X = tf.placeholder(tf.float32, [None, 784], name="X")
            # Y_: 标签
            Y_ = tf.placeholder(tf.float32, [None, 10])

        # L1：200个神经元
        Y1 = add_layer(X, 784, 200, tf.nn.sigmoid)

        # L2：100个神经元
        Y2 = add_layer(Y1, 200, 100, tf.nn.sigmoid)

        # L3: 60个神经元
        Y3 = add_layer(Y2, 100, 60, tf.nn.sigmoid)

        # L4: 30个神经元
        Y4 = add_layer(Y3, 60, 30, tf.nn.sigmoid)

        # L5: 10个神经元
        Ylogits = add_layer(Y4, 30, 10, tf.nn.sigmoid)

        # Output
        Y = tf.nn.softmax(Ylogits)

        # 损失函数
        with tf.name_scope('loss'):
            cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y_, logits=Ylogits))

        tf.summary.scalar('cross_entropy', cross_entropy)

        # 优化算法
        with tf.name_scope('train'):
            train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

        # 计算准确率
        with tf.name_scope('accuracy'):
            correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        tf.summary.scalar('accuracy', accuracy)

        merged = tf.summary.merge_all()

        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            # 将当前会话中的计算图保存
            writer = tf.summary.FileWriter("logs/adm_lr_{0}/".format(lr), sess.graph)

            for i in range(10000):
                batch_xs, batch_ys = mnist.train.next_batch(100)
                summary, _ = sess.run([merged, train_step], feed_dict={X: batch_xs, Y_: batch_ys})

                writer.add_summary(summary, i)

                if i%100 == 0:
                    print(accuracy.eval({X: mnist.test.images, Y_: mnist.test.labels}))

/Users/lyon/.pyenv/versions/3.6.4/envs/scikit-learn/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
0.0998
0.472
0.6761
0.7672
0.8201
0.8426
0.8665
0.8788
0.8868
0.8965
0.903
0.9055
0.9106
0.9149
0.9139
0.9176
0.9176
0.9214
0.9248
0.9247
0.9266
0.9286
0.9298
0.9296
0.9306
0.9332
0.9323
0.9341
0.9352
0.9367
0.9347
0.9366
0.9371
0.9376
0.939
0.9406
0.9411
0.9409
0.9427
0.941
0.9428
0.9418
0.9427
0.94

0.9596
0.9524
0.9552
0.9596
0.9554
0.9556
0.9604
0.9573
0.9536
0.9555
0.9564
0.9557
0.959
0.9572
0.9575
0.9554
0.9568
0.9556
0.9531
0.9588
0.9565
0.9603
0.9592
0.9581
0.9631
0.9615
0.9603
0.9613
0.9606
0.9558
0.9583
0.9634
0.9584
0.9587
0.9591
0.9585
0.9547
0.958
0.9561
0.9618
0.9577
0.9622
0.9596
0.9562
0.9528
0.9549
0.9579
0.9604
0.9562
0.9634
0.9613
0.9618
0.9571
0.9612
0.1029
0.7972
0.9
0.916
0.9239
0.9286
0.9288
0.9353
0.9349
0.9392
0.9359
0.9361
0.9415
0.9467
0.9464
0.942
0.9508
0.9434
0.9433
0.9515
0.9425
0.9509
0.9521
0.9512
0.9521
0.9541
0.9542
0.9544
0.9506
0.9555
0.9517
0.9546
0.9408
0.9485
0.9477
0.9469
0.9485
0.9551
0.95
0.9516
0.9542
0.9496
0.9563
0.9534
0.9539
0.9467
0.9508
0.9522
0.9529
0.9543
0.9553
0.956
0.9559
0.9547
0.9542
0.9482
0.9596
0.9552
0.9553
0.9479
0.9537
0.9572
0.9547
0.9526
0.957
0.9523
0.955
0.9559
0.9497
0.95
0.9569
0.9511
0.953
0.9564
0.9536
0.9511
0.9557
0.948
0.9528
0.9555
0.9504
0.9545
0.9587
0.9543
0.9511
0.9602
0.9588
0.9526
0.9518
0.958
0.954
0.9

# 3. 对比不同算法的性能 (Optional)

**练习**: 对比`GradientDescentOptimizer`和`tf.train.AdamOptimizer`的性能区别。
提示：使用GradientDescentOptimizer算法，学习速率从0.1开始一直到1.0，输出到子目录`logs/gd_lr_{value}`下。

# 4. 本地安装TF环境

首先安装Python 3的环境，然后通过pip进行安装
```
pip install tensorflow
pip install jupyter
```

安装完成后，启动jupyter服务端：
```
jupyter notebook
```